In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append('../')
os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3,4,5,6,7'
os.environ['DATA'] = "/datascope/subaru/user/swei20/huggingface/datasets/"

In [3]:
# os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3,4,5,6,7'
import numpy as np
import datetime
import argparse
import torch
# torch.cuda.device_count(), torch.cuda.is_available()
from transformers import AutoModelForSequenceClassification,  TrainingArguments, Trainer 
from peft import get_peft_model, LoraConfig, TaskType
import evaluate


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/swei20/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /datascope/slurm/miniconda3/envs/viska-torch/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/swei20/.local/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113_nocublaslt.so...


/home/swei20/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/datascope/slurm/miniconda3/envs/viska-torch/lib/libcudart.so'), PosixPath('/datascope/slurm/miniconda3/envs/viska-torch/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
/home/swei20/.local/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [4]:
import src.util as util

In [6]:
token = "xlmr1"
tokenizer = util.get_tokenizer(token)
ds = util.get_ds(tokenizer)

Found cached dataset arrow (/home/swei20/.cache/huggingface/datasets/arrow/emotion-99eb813a81a18198/0.0.0/74f69db2c14c2860059d39860b1f400a03d11bf7fb5a8258ca38c501c878c137)


  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [7]:
model = AutoModelForSequenceClassification.from_pretrained(util.TOKEN[token],
                                                        #    device_map=device_map,
                                                        torch_dtype=torch.bfloat16,
                                                        num_labels=6,)

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier

In [8]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, out_fe

In [25]:
peft_config = LoraConfig(
        task_type=TaskType.SEQ_CLS, 
        inference_mode=False, 
        r=2048, 
        lora_alpha=4096, 
        lora_dropout=0.1,
        bias='all',
        # bias='none',
        target_modules=['query', 'key', 'value', 'dense'],
    )

In [26]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 914631692 || all params: 1475310604 || trainable%: 61.995873243245526


In [2]:
from transformers import AutoTokenizer
model_path = "/home/swei20/NLP/hfmodels/xlmr.base"
tokenizer = AutoTokenizer.from_pretrained(model_path, checkpoint_file='model.pt')

OSError: /home/swei20/NLP/hfmodels/xlmr.base does not appear to have a file named config.json. Checkout 'https://huggingface.co//home/swei20/NLP/hfmodels/xlmr.base/None' for available files.

In [3]:
from transformers import AutoTokenizer, XLMRobertaXLModel

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-xlarge")
# model = XLMRobertaXLModel.from_pretrained("xlm-roberta-xlarge")

OSError: xlm-roberta-xlarge is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [4]:
from transformers import XLMRobertaXLConfig, XLMRobertaXLModel

# Initializing a XLM_ROBERTA_XL bert-base-uncased style configuration
configuration = XLMRobertaXLConfig()

# Initializing a model (with random weights) from the bert-base-uncased style configuration
model = XLMRobertaXLModel(configuration)

# Accessing the model configuration
configuration = model.config

In [7]:
ckpt = "/home/swei20/NLP/hfmodels/xlmr.base/model.pt"

In [11]:
import torch
xlmr = torch.hub.load(model_path, 'xlmr.large')

ValueError: too many values to unpack (expected 2)

In [8]:
# model_path = "/home/swei20/NLP/hfmodels/xlmr.base/"
model._load_pretrained_model(ckpt)

# ()

TypeError: _load_pretrained_model() missing 4 required positional arguments: 'state_dict', 'loaded_keys', 'resolved_archive_file', and 'pretrained_model_name_or_path'